In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
DATA_PATH = "/content/drive/MyDrive/Colab Notebooks/04.ML_Competition/data/"
SEED = 42

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import GradientBoostingClassifier

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
!pip install catboost
!pip install category_encoders
import category_encoders as ce

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import SelectPercentile

from scipy.stats import skew

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
trans_train = pd.read_csv(f"{DATA_PATH}trans_train.csv") # 학습 데이터
ft_train = pd.read_csv(f"{DATA_PATH}target_train.csv") # 학습데이터 정답
trans_test = pd.read_csv(f"{DATA_PATH}trans_test.csv") # 테스트 데이터
sample_submission = pd.read_csv(f"{DATA_PATH}sample_submission.csv") # 제출파일

trans_train.shape, ft_train.shape, trans_test.shape, sample_submission.shape

((167465, 7), (2554, 2), (64539, 7), (946, 2))

In [10]:
ft_test = sample_submission[["고객번호"]]

trans_train["구매일자"] = pd.to_datetime(trans_train["구매일자"]) 
trans_test["구매일자"] = pd.to_datetime(trans_test["구매일자"])

ft_test.head()

,고객번호
0,10001
1,10070
2,10075
3,10086
4,10128


In [11]:
# 여성_대분류
female_large = ["디자이너","란제리/내의","멀티(아울렛)","생활잡화","섬유잡화","소형가전","식기","아동","주방용품","차/커피","축산가공","침구/수예","모피/피혁", "보석","화장품"]
# 남성_대분류
male_large = ["남성캐주얼","남성 트랜디","남성 트랜디","남성정장","셔츠"]
# 여성_중분류
female_mid = ['보석', '란제리','스카프(특정)','NB 여화','숙녀고정행사','기초 화장품','유아용품','스타킹(특정)','유아복','수입ACC','색조 화장품','숙녀단기행사','스카프(단기행사)','란제리행사',  '크리스탈', '국산ACC','직수입핸드백',  '헤어ACC', '14K','수예단기행사', '명품잡화보석','명품보석', '수입잡화보석','란제리(멀티)', '머플러(특정)']
# 남성_중분류
male_mid = ['(특정)','명품남성', '넥타이(단기행사)', '맞춤셔츠','명품시계기타','명품시계직매입']
# 지점_수도권 13
capital_area = [ '강남점', '본  점', '잠실점', '미아점', '인천점',
                '영등포점', '부평점', '분당점', '일산점', '노원점', '관악점', '청량리점', '안양점' ]
# 지점_지방 11
province = [ '부산본점', '센텀시티점', '울산점', '대전점', '광주점', '동래점',
            '창원점', '대구점', '전주점', '포항점', '상인점' ]
#식품
food = ["주류", "가공식품", "수산품", "농산물", "축산가공", "육류", "차/커피", "젓갈/반찬", "건강식품"] 

#옷, 장신구, 기타 잡화
obj = ["캐주얼", "피혁잡화","기타", "섬유잡화", "생활잡화", "명품",
       "남성정장", "보석", "셔츠", "구두", "란제리/내의","트레디셔널", "남성 트랜디","식기",
       "주방용품", "골프", "커리어", "스포츠", "디자이너", "시티웨어", "기타의류", "모피/피혁", "화장품", "남성캐주얼",
       "아동", "액세서리", "일용잡화", "문구/팬시", "준보석/시계", "악기", "교복", "웨딩", "멀티(아울렛)", "전자/전기"]
       
# 부피가 크고 비쌀 것으로 예상되는 가전제품, 가구 등
app = ["대형가전", "주방가전", "소형가전", "통신/컴퓨터", "가구", "침구/수예"]

# 주부_중분류
wife_mid = ["란제리","수산","스카프(특정)","밥솥",'김류', '멸치류', '밥류', '일식델리', '유제품', '우육', '양념육','숙녀고정행사', '생선','유아용품',
        '햄', '농산가공', '유기농야채','야채', '청과', '디저트류', '유아복','단기행사(아동복)', 'N/B 아동복',
      'L/C 아동복','돈육','수입육','수입주방','카페트', '단기행사', '가구 단독매입', '위생세제',
      '기타생선','곡물', '규격김치', '타월', '규격젓갈', '계육', '장류', '벌크김치','스카프(단기행사)', '국산주방', '수예소품',
       '생활잡화 단독매입', '란제리행사', '주방잡화', '욕실용품', '아동침구', '고정행사(아동복)', '소파', '즉석반찬', '휴지류', '냉동식품', '중식델리',
       '어묵', '크리스탈', '즉석구이김류','직수입핸드백', '헤어ACC', '정육세트', '14K', '식탁', '한과(세트행사)', '화장품', '가 방', '아동신발',  '특수야채', '수예단기행사',
        '명품잡화보석', '주방단기행사', '교복', '수입캐릭터',
        '기능성침대',  '아동잡화', '수입잡화보석','란제리(멀티)', '신생아완구 단독매입']
# 엄마_중분류
mom_mid = ['유아용품','유아복','단기행사(아동복)', 'N/B 아동복','L/C 아동복','아동침구','고정행사(아동복)','아동잡화','수입캐릭터','신생아완구 단독매입']
# 주부_대분류
wife_large = ['침구/수예','생활잡화','수산품','식기', '주방용품','농산물','주방가전','육류','젓갈/반찬']

# 상세한 남자 용품
detail_male = ["정장,넥타이,남성,골프,스포츠,시티,셔츠,신사,시계,남화"]
# 상세한 여자 용품
detail_female = ["보석,여화,스카프,숙녀,화장품,핸드백,뷰티,란제리,엘리든,머플러"]


In [12]:
agg_dict = {
    '구매일자':[
        ("내점일수","nunique"),
        ("구매주기",lambda x : (x.max() - x.min()).days / x.nunique() ),
        ("주말방문횟수", lambda x : (x.dt.weekday > 4).mean() ),
        ("평일방문횟수", lambda x : (x.dt.weekday < 5).count() ),
        ("주말방문비율", lambda x : (x.dt.weekday.isin([5,6])).sum() / x.nunique() ),
        ("평일방문비율", lambda x : (x.dt.weekday.isin([1,2,3,4])).sum() / x.nunique() ),
        ("주구매요일" , lambda x: x.dt.weekday.mode()[0] ),
        ("봄구매건수" , lambda x : (x.dt.month.isin([3,4,5])).sum() ),
        ("여름구매건수" , lambda x : (x.dt.month.isin([6,7,8])).sum() ),
        ("가을구매건수" , lambda x : (x.dt.month.isin([9,10,11])).sum() ),
        ("겨울구매건수" , lambda x : (x.dt.month.isin([12,1,2])).sum() ),
        ("봄구매비율" , lambda x : (x.dt.month.isin([3,4,5])).mean() ),
        ("여름구매비율" , lambda x : (x.dt.month.isin([6,7,8])).mean() ),
        ("가을구매비율" , lambda x : (x.dt.month.isin([9,10,11])).mean() ),
        ("겨울구매비율" , lambda x : (x.dt.month.isin([12,1,2])).mean() ),
        ("일평균구매건수" , lambda x : x.count() / x.nunique() ),
        ("거래개월수" , lambda x : x.dt.month.nunique() ),
        ("주구매개월", lambda x : x.dt.month.mode()[0] ),
        ("간단방문비율",lambda x: (x.value_counts()<3).mean()),
        ("장보기비율",lambda x: (x.value_counts()>10).mean()),
        ("월초구매비율", lambda x : (x.dt.day<6).mean()),
        ("월말구매비율", lambda x : (x.dt.day>25).mean()),
        ("월요일구매건수", lambda x : (x.dt.weekday == 0).sum()),
        ("화요일구매건수", lambda x : (x.dt.weekday == 1).sum()),
        ("수요일구매건수", lambda x : (x.dt.weekday == 2).sum()),
        ("목요일구매건수", lambda x : (x.dt.weekday == 3).sum()),
        ("금요일구매건수", lambda x : (x.dt.weekday == 4).sum()),
        ("토요일구매건수", lambda x : (x.dt.weekday == 5).sum()),
        ("월요일구매비율", lambda x : (x.dt.weekday == 0).mean()),
        ("화요일구매비율", lambda x : (x.dt.weekday == 1).mean()),
        ("수요일구매비율", lambda x : (x.dt.weekday == 2).mean()),
        ("목요일구매비율", lambda x : (x.dt.weekday == 3).mean()),
        ("금요일구매비율", lambda x : (x.dt.weekday == 4).mean()),
        ("토요일구매비율", lambda x : (x.dt.weekday == 5).mean()),

    ],
    '지점':[
         ("방문지점수","nunique"),
         ("주구매지점" , lambda x : x.mode()[0]),
         ("방문지점다양성", lambda x: x.nunique() / x.count() ),
         ("수도권방문횟수", lambda x : x.isin(capital_area).sum()),
         ("수도권방문비율", lambda x : x.isin(capital_area).mean()),
         ("지방방문횟수", lambda x : x.isin(province).sum()),
         ("지방방문비율", lambda x : x.isin(province).mean())
    ],
    '상품번호': [
             ('상품번호수', 'nunique'),
             ("구매상품다양성",lambda x: x.nunique() / x.count() )
             ],
    '중분류': [
            ('중분류수', 'nunique'),
            ('주구매중분류', lambda x: x.mode()[0]),
            ("중_여성제품구매비율",lambda x: (x.isin(female_mid)).mean()),
            ("중_남성제품구매비율",lambda x: (x.isin(male_mid)).mean()),
            ("중_여성제품구매건수",lambda x: (x.isin(female_mid)).sum()),
            ("중_남성제품구매건수",lambda x: (x.isin(male_mid)).sum()),
            ("중_주부관련구매비율",lambda x: (x.isin(wife_mid)).mean()),
            ("중_주부관련구매횟수",lambda x: (x.isin(wife_mid)).sum()),

            ("중분류다양성", lambda x : x.nunique() / x.count()),
            ("알뜰지수비율", lambda x: (sum(x.str.contains("행사", regex = True))) / x.count()),
            ("알뜰지수횟수", lambda x: (sum(x.str.contains("행사", regex = True))))
            ],
    '대분류': [ 
            ('대분류수', 'nunique'),
            ('주구매대분류', lambda x: x.mode()[0]),
            ("대_여성제품구매비율",lambda x: (x.isin(female_large)).mean()),
            ("대_남성제품구매비율",lambda x: (x.isin(male_large)).mean()),
            ("대_여성제품구매건수",lambda x: (x.isin(female_large)).sum()),
            ("대_남성제품구매건수",lambda x: (x.isin(male_large)).sum()),
            ("대_주부제품구매비율",lambda x: (x.isin(wife_large)).mean()),
            ("대_주부제품구매횟수",lambda x: (x.isin(wife_large)).sum()),
            ("대분류다양성", lambda x : x.nunique() / x.count()),
            ("아동구매건수", lambda x : x.str.contains("아동|유아").count()),
            ("아동구매비율", lambda x : x.str.contains("아동|유아").count() / x.nunique()),
            ("남성의류구매건수", lambda x: (x.str.contains("남성|시티웨어|셔츠").count())),
            ("스포츠관련구매건수", lambda x: (x.str.contains("골프|스포츠").count())),
            ("여성용품구매건수", lambda x :(x.str.contains("란제리|화장품|보석|구두|액세서리|웨딩|모피|차|명품").count())),
            ("남성의류제외구매건수", lambda x: (x.str.contains("캐주얼|디자이너|기타의류").count())),
            ("식료품구매 건수", lambda x:(x.str.contains("식품|수산|축산|육류|반찬|커피|차|건강").count())),
            ("식료품구매 비율", lambda x:(x.str.contains("식품|수산|축산|육류|반찬|커피|차|건강").count()) / x.nunique()),
            ("가전,가구구매 건수", lambda x:(x.str.contains("가전|가구|침구").count())),
            ("전자제품구매 건수", lambda x:(x.str.contains("통신|전기").count())),
            ("자식용품구매 건수", lambda x:(x.str.contains("아동|교복|문구")).count())
            ],
    '구매가격' : [
        ('총구매액','sum'),
        ('구매건수', 'count'), 
        ('평균구매액', "mean"),
        ('최대구매액', 'max'),
        ('최소구매액',lambda x: x[x > 0].min() ) ,
        ('구매금액표준편차',lambda x: x[x>0].std() ),
    ],
}

In [13]:
tmp = trans_train.groupby('고객번호').agg(agg_dict)
tmp.columns = tmp.columns.droplevel()
tmp = tmp.reset_index()

ft_train = ft_train.merge(tmp,how="left",on = "고객번호")
ft_train

,고객번호,target,내점일수,구매주기,주말방문횟수,평일방문횟수,주말방문비율,평일방문비율,주구매요일,봄구매건수,...,식료품구매 비율,"가전,가구구매 건수",전자제품구매 건수,자식용품구매 건수,총구매액,구매건수,평균구매액,최대구매액,최소구매액,구매금액표준편차
0,10004,1,26,11.423077,0.617647,68,1.615385,0.923077,5,11,...,2.720000,68,68,68,390166360,68,5.737741e+06,41760000,6570.0,9.453819e+06
1,10013,0,11,21.090909,0.117647,17,0.181818,1.363636,4,6,...,1.888889,17,17,17,22447800,17,1.320459e+06,13375000,6900.0,3.179888e+06
2,10023,0,21,16.666667,0.348837,43,0.714286,1.190476,3,15,...,2.529412,43,43,43,81765810,43,1.901530e+06,14325000,1250.0,3.738478e+06
3,10040,0,24,13.833333,0.121212,66,0.333333,1.750000,4,18,...,4.400000,66,66,66,213705680,66,3.237965e+06,24738000,1500.0,5.883481e+06
4,10052,0,28,8.892857,0.333333,111,1.321429,2.464286,2,65,...,5.550000,111,111,111,358336570,111,3.228257e+06,38328000,1250.0,6.959274e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2549,49813,0,83,4.253012,0.167116,371,0.746988,3.180723,1,84,...,18.550000,371,371,371,47428060,371,1.278384e+05,9120000,1000.0,5.931527e+05
2550,49892,0,31,10.096774,0.363636,121,1.419355,2.096774,6,22,...,6.050000,121,121,121,54369820,121,4.493374e+05,7500000,1100.0,1.148466e+06
2551,49925,0,3,18.333333,0.333333,3,0.333333,0.666667,3,0,...,3.000000,3,3,3,1657400,3,5.524667e+05,1015000,39000.0,4.899894e+05
2552,49943,0,7,30.285714,0.545455,11,0.857143,0.571429,5,1,...,1.571429,11,11,11,40742400,11,3.703855e+06,21780000,36000.0,6.610017e+06


In [14]:
tmp = trans_test.groupby('고객번호').agg(agg_dict)
tmp.columns = tmp.columns.droplevel()
tmp = tmp.reset_index()

ft_test = ft_test.merge(tmp,how="left",on = "고객번호")

In [15]:
ft_train.shape, ft_test.shape

((2554, 82), (946, 81))

In [16]:
# 대분류 분석을 통해 아이가 있는 가정인지 아닌지 알 수 있다.

agg_list = [
    ('아동개수' , lambda x : x[x.str.contains('아동')].count()) # 대분류에 있는 아동 count
]

tmp = trans_train.groupby('고객번호')['대분류'].agg(agg_list).reset_index()

num = ['O' if x > 2 else 'X' for x in tmp['아동개수']] # 아동이 0보다 많으면 O를 아니면 X를 반환
tmp['아이여부'] = num
tmp = tmp.drop(columns = ['아동개수'])
tmp
ft_train = ft_train.merge(tmp, how='left')
ft_train.head()

,고객번호,target,내점일수,구매주기,주말방문횟수,평일방문횟수,주말방문비율,평일방문비율,주구매요일,봄구매건수,...,"가전,가구구매 건수",전자제품구매 건수,자식용품구매 건수,총구매액,구매건수,평균구매액,최대구매액,최소구매액,구매금액표준편차,아이여부
0,10004,1,26,11.423077,0.617647,68,1.615385,0.923077,5,11,...,68,68,68,390166360,68,5.737741e+06,41760000,6570.0,9.453819e+06,X
1,10013,0,11,21.090909,0.117647,17,0.181818,1.363636,4,6,...,17,17,17,22447800,17,1.320459e+06,13375000,6900.0,3.179888e+06,X
2,10023,0,21,16.666667,0.348837,43,0.714286,1.190476,3,15,...,43,43,43,81765810,43,1.901530e+06,14325000,1250.0,3.738478e+06,X
3,10040,0,24,13.833333,0.121212,66,0.333333,1.750000,4,18,...,66,66,66,213705680,66,3.237965e+06,24738000,1500.0,5.883481e+06,O
4,10052,0,28,8.892857,0.333333,111,1.321429,2.464286,2,65,...,111,111,111,358336570,111,3.228257e+06,38328000,1250.0,6.959274e+06,O


In [17]:
# 대분류 분석을 통해 아이가 있는 가정인지 아닌지 알 수 있다.

agg_list = [
    ('아동개수' , lambda x : x[x.str.contains('아동')].count()) # 대분류에 있는 아동 count
]

tmp = trans_test.groupby('고객번호')['대분류'].agg(agg_list).reset_index()

num = ['O' if x > 2 else 'X' for x in tmp['아동개수']] # 아동이 0보다 많으면 O를 아니면 X를 반환
tmp['아이여부'] = num
tmp = tmp.drop(columns = ['아동개수'])
tmp
ft_test = ft_test.merge(tmp, how="left", on="고객번호")
ft_test.head()

,고객번호,내점일수,구매주기,주말방문횟수,평일방문횟수,주말방문비율,평일방문비율,주구매요일,봄구매건수,여름구매건수,...,"가전,가구구매 건수",전자제품구매 건수,자식용품구매 건수,총구매액,구매건수,평균구매액,최대구매액,최소구매액,구매금액표준편차,아이여부
0,10001,2,48.0,0.400000,10,2.0,3.00,3,0,4,...,10,10,10,3048890,10,3.048890e+05,1197600,19000,3.921522e+05,X
1,10070,10,21.5,0.250000,28,0.7,1.40,0,7,6,...,28,28,28,12803700,28,4.572750e+05,4320000,4000,9.659565e+05,X
2,10075,4,12.5,0.285714,7,0.5,1.25,3,3,0,...,7,7,7,9546000,7,1.363714e+06,2880000,90000,1.212731e+06,X
3,10086,2,7.0,0.000000,4,0.0,2.00,4,0,0,...,4,4,4,4995000,4,1.248750e+06,2646000,20000,1.220515e+06,X
4,10128,1,0.0,0.000000,1,0.0,1.00,2,0,0,...,1,1,1,650000,1,6.500000e+05,650000,650000,NaN,X


In [18]:
#월 평균 구매 빈도
agg_list = [
    ("월평균구매빈도", lambda x : x.count() / x.dt.month.nunique() )
]
tmp = trans_train.groupby("고객번호")["구매일자"].agg(agg_list).reset_index()
ft_train = ft_train.merge(tmp, how="left", on="고객번호")
ft_train.head()

,고객번호,target,내점일수,구매주기,주말방문횟수,평일방문횟수,주말방문비율,평일방문비율,주구매요일,봄구매건수,...,전자제품구매 건수,자식용품구매 건수,총구매액,구매건수,평균구매액,최대구매액,최소구매액,구매금액표준편차,아이여부,월평균구매빈도
0,10004,1,26,11.423077,0.617647,68,1.615385,0.923077,5,11,...,68,68,390166360,68,5.737741e+06,41760000,6570.0,9.453819e+06,X,7.555556
1,10013,0,11,21.090909,0.117647,17,0.181818,1.363636,4,6,...,17,17,22447800,17,1.320459e+06,13375000,6900.0,3.179888e+06,X,3.400000
2,10023,0,21,16.666667,0.348837,43,0.714286,1.190476,3,15,...,43,43,81765810,43,1.901530e+06,14325000,1250.0,3.738478e+06,X,3.909091
3,10040,0,24,13.833333,0.121212,66,0.333333,1.750000,4,18,...,66,66,213705680,66,3.237965e+06,24738000,1500.0,5.883481e+06,O,6.600000
4,10052,0,28,8.892857,0.333333,111,1.321429,2.464286,2,65,...,111,111,358336570,111,3.228257e+06,38328000,1250.0,6.959274e+06,O,12.333333


In [19]:
#월 평균 구매 빈도
agg_list = [
    ("월평균구매빈도", lambda x : x.count() / x.dt.month.nunique() )
]
tmp = trans_test.groupby("고객번호")["구매일자"].agg(agg_list).reset_index()
ft_test = ft_test.merge(tmp, how="left", on="고객번호")
ft_test.head()

,고객번호,내점일수,구매주기,주말방문횟수,평일방문횟수,주말방문비율,평일방문비율,주구매요일,봄구매건수,여름구매건수,...,전자제품구매 건수,자식용품구매 건수,총구매액,구매건수,평균구매액,최대구매액,최소구매액,구매금액표준편차,아이여부,월평균구매빈도
0,10001,2,48.0,0.400000,10,2.0,3.00,3,0,4,...,10,10,3048890,10,3.048890e+05,1197600,19000,3.921522e+05,X,5.000000
1,10070,10,21.5,0.250000,28,0.7,1.40,0,7,6,...,28,28,12803700,28,4.572750e+05,4320000,4000,9.659565e+05,X,4.666667
2,10075,4,12.5,0.285714,7,0.5,1.25,3,3,0,...,7,7,9546000,7,1.363714e+06,2880000,90000,1.212731e+06,X,2.333333
3,10086,2,7.0,0.000000,4,0.0,2.00,4,0,0,...,4,4,4995000,4,1.248750e+06,2646000,20000,1.220515e+06,X,4.000000
4,10128,1,0.0,0.000000,1,0.0,1.00,2,0,0,...,1,1,650000,1,6.500000e+05,650000,650000,NaN,X,1.000000


In [20]:
#고객별 매점당 구매건수 완료

df1 = trans_train.pivot_table(
    index = "고객번호",
    columns = "지점",
    values = "상품번호",
    aggfunc = "count" #상품번호의 카운트가 구매건수이다.
).reset_index()
df1 = df1.add_suffix("_구매건수") #전체 컬럼 옆에_구매건수로 컬럼명을 바꾼다.
df1 = df1.rename(columns={"고객번호_구매건수":"고객번호"}) #고객번호는 남겨놔야 하므로 제대로 바꿔준다.
df1 = df1.fillna(0) #해당 지점의 구매건수가 없다면 결측치를 0으로 바꿔준다.
ft_train = ft_train.merge(df1, how="left")

In [21]:
#매점당 구매금액 완료
df1 = pd.pivot_table(
    trans_train,
    index = "고객번호",
    columns = "지점",
    values = "구매가격",
    aggfunc = "sum" #고객별 지점별 구매금액의 합계
).reset_index()
df1 = df1.add_suffix("_구매금액")
df1 = df1.rename(columns={"고객번호_구매금액":"고객번호"})
df1 = df1.fillna(0)
ft_train = ft_train.merge(df1, how="left")

In [22]:
#고객별 매점당 구매건수 완료

df1 = trans_train.pivot_table(
    index = "고객번호",
    columns = "지점",
    values = "상품번호",
    aggfunc = "count" #상품번호의 카운트가 구매건수이다.
).reset_index()
df1 = df1.add_suffix("_구매건수") #전체 컬럼 옆에_구매건수로 컬럼명을 바꾼다.
df1 = df1.rename(columns={"고객번호_구매건수":"고객번호"}) #고객번호는 남겨놔야 하므로 제대로 바꿔준다.
df1 = df1.fillna(0) #해당 지점의 구매건수가 없다면 결측치를 0으로 바꿔준다.
ft_test = ft_test.merge(df1, how="left")

In [23]:
#매점당 구매금액 완료
df1 = pd.pivot_table(
    trans_train,
    index = "고객번호",
    columns = "지점",
    values = "구매가격",
    aggfunc = "sum" #고객별 지점별 구매금액의 합계
).reset_index()
df1 = df1.add_suffix("_구매금액")
df1 = df1.rename(columns={"고객번호_구매금액":"고객번호"})
df1 = df1.fillna(0)
ft_test = ft_test.merge(df1, how="left")

In [24]:
# 1,2,9월
agg_dict = {
    '구매일자':[
        ('1|2|9월횟수',lambda x : (x.dt.month.isin([1,2,9])).sum() ) 
    ]
}

tr = trans_train.groupby('고객번호').agg(agg_dict)
tr.columns = tr.columns.droplevel()
tr = tr.reset_index()
ft_train = ft_train.merge(tr[['고객번호','1|2|9월횟수']],how="left",on="고객번호")

In [25]:
tr = trans_test.groupby('고객번호').agg(agg_dict)
tr.columns = tr.columns.droplevel()
tr = tr.reset_index()

ft_test = ft_test.merge(tr[['고객번호','1|2|9월횟수']],how="left",on = "고객번호")

In [26]:
agg_dict = {
    '구매가격':[
        ('총구매가격','sum') # 환불은 제외해야 하므로 음수 제외하지 않고 그대로 sum
    ]
}
tr = trans_train.groupby('고객번호').agg(agg_dict)
tr.columns = tr.columns.droplevel()
tr = tr.reset_index()
#  - 400만원 이상: Vip -> 200
#  - 800만원 이상: Vip+ -> 300
#  - 1800~2000만원이상: MVG-Ace  -> 400
#  - 4000만원이상: MVG-Crown -> 500
#  - 6000만원이상: MVG-Prestige -> 600
#  - 1억이상: LENITH -> 700
#  - 해당하지 않는 고객들은 silver로 설정 -> 100
def money(x):

    if 4000000<= x['총구매가격'] < 8000000:
        return 'vip'
    elif 8000000<= x['총구매가격'] < 18000000:
        return 'vip+'
    elif 18000000<= x['총구매가격'] <40000000:
        return 'MVG-Ace'
    elif 40000000<= x['총구매가격'] <60000000:
        return 'MVG-Crown'
    elif 60000000<= x['총구매가격'] <100000000:
        return 'MVG-Prestige'
    elif x['총구매가격']>= 100000000:
        return 'lenith'
    else:
        return 'silver'

tr['고객등급'] = tr.apply(money, axis=1)
ft_train = ft_train.merge(tr[['고객번호','고객등급']],how="left",on = "고객번호")

In [27]:
tr = trans_test.groupby('고객번호').agg(agg_dict)
tr.columns = tr.columns.droplevel()
tr = tr.reset_index()
tr['고객등급'] = tr.apply(money, axis=1) # 이거는 확인 필요함.
ft_test = ft_test.merge(tr[['고객번호','고객등급']],how="left",on = "고객번호")

# pivot feature 추가

## 고객별 중분류 구매 건수

In [28]:
train_tmp = pd.pivot_table(trans_train,index="고객번호",columns="중분류",values="구매가격",aggfunc="count",fill_value=0).reset_index()
ft_train = ft_train.merge(train_tmp,how="left",on="고객번호")

In [29]:
test_tmp = pd.pivot_table(trans_test,index="고객번호",columns="중분류",values="구매가격",aggfunc="count",fill_value=0).reset_index()
test_tmp.head()

중분류,고객번호,14K,4대 B/D,"IT 게임기,S/W",KKD,L.B,L/C 아동복,L/C골프의류,L/C정장,MP3 外,...,한실침구,한차,햄,햄세트,향수,헤어ACC,홈웨어,화장품,회,휴지류
0,10001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10070,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,10075,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10086,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10128,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
for col in train_tmp.columns:
    if col not in test_tmp.columns:# 학습피쳐에는 있으나 테스트피쳐에는 없는 컬럼
        test_tmp[col] = 0 # 0으로 채우기

# 학습피쳐와 컬럼및 컬럼 순서 맞추기
test_tmp = test_tmp[train_tmp.columns]

In [31]:
ft_test = ft_test.merge(test_tmp,how="left",on="고객번호")

## 고객별 중분류 구매 건수

In [32]:
train_tmp = pd.pivot_table(trans_train,index="고객번호",columns="대분류",values="구매가격",aggfunc="count",fill_value=0).reset_index()
ft_train = ft_train.merge(train_tmp,how="left",on="고객번호")
ft_train.head()

,고객번호,target,내점일수,구매주기,주말방문횟수,평일방문횟수,주말방문비율,평일방문비율,주구매요일,봄구매건수,...,준보석/시계,차/커피,축산가공,침구/수예,캐주얼,커리어_y,통신/컴퓨터,트래디셔널_y,피혁잡화,화장품_y
0,10004,1,26,11.423077,0.617647,68,1.615385,0.923077,5,11,...,0,0,0,2,5,1,0,4,1,0
1,10013,0,11,21.090909,0.117647,17,0.181818,1.363636,4,6,...,0,0,0,0,0,0,0,0,0,0
2,10023,0,21,16.666667,0.348837,43,0.714286,1.190476,3,15,...,0,1,1,0,0,0,0,1,0,1
3,10040,0,24,13.833333,0.121212,66,0.333333,1.750000,4,18,...,0,1,5,0,2,0,0,0,0,6
4,10052,0,28,8.892857,0.333333,111,1.321429,2.464286,2,65,...,0,0,1,1,1,2,0,0,0,5


In [33]:
test_tmp = pd.pivot_table(trans_test,index="고객번호",columns="대분류",values="구매가격",aggfunc="count",fill_value=0).reset_index()
test_tmp.head()

대분류,고객번호,가공식품,가구,건강식품,골프,교복,구두,기타,기타의류,남성 캐주얼,...,준보석/시계,차/커피,축산가공,침구/수예,캐주얼,커리어,통신/컴퓨터,트래디셔널,피혁잡화,화장품
0,10001,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10070,8,0,0,0,0,0,6,0,0,...,0,1,2,2,0,0,0,0,0,0
2,10075,0,0,0,0,0,0,3,0,0,...,0,0,0,0,0,0,0,0,1,3
3,10086,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10128,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
for col in train_tmp.columns:
    if col not in test_tmp.columns:# 학습피쳐에는 있으나 테스트피쳐에는 없는 컬럼
        test_tmp[col] = 0 # 0으로 채우기

# 학습피쳐와 컬럼및 컬럼 순서 맞추기
test_tmp = test_tmp[train_tmp.columns]
test_tmp.head()
ft_test = ft_test.merge(test_tmp, how="left", on="고객번호")
ft_test.head()

,고객번호,내점일수,구매주기,주말방문횟수,평일방문횟수,주말방문비율,평일방문비율,주구매요일,봄구매건수,여름구매건수,...,준보석/시계,차/커피,축산가공,침구/수예,캐주얼,커리어_y,통신/컴퓨터,트래디셔널_y,피혁잡화,화장품_y
0,10001,2,48.0,0.400000,10,2.0,3.00,3,0,4,...,0,0,0,0,0,0,0,0,0,0
1,10070,10,21.5,0.250000,28,0.7,1.40,0,7,6,...,0,1,2,2,0,0,0,0,0,0
2,10075,4,12.5,0.285714,7,0.5,1.25,3,3,0,...,0,0,0,0,0,0,0,0,1,3
3,10086,2,7.0,0.000000,4,0.0,2.00,4,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10128,1,0.0,0.000000,1,0.0,1.00,2,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
ft_train.shape , ft_test.shape

((2554, 500), (946, 499))

# 결측치

In [36]:
ft_train = ft_train.fillna(0)
ft_test = ft_test.fillna(0)

ft_train.isnull().sum().sum() , ft_test.isnull().sum().sum()

(0, 0)

# 학습 , 정답, 테스트, 수치형&변수형 변수 분리, 스케일링

In [37]:
train = ft_train.iloc[:,2:]
target = ft_train.iloc[:,1]

test = ft_test.iloc[:,1:]

In [38]:
cat_cols = ft_train.dtypes[ft_train.dtypes == "object"].index.tolist()
num_cols = train.select_dtypes(exclude='object').columns.to_list()

In [39]:
test_cat_cols = ft_test.dtypes[ft_test.dtypes == "object"].index.tolist()
test_num_cols = test.select_dtypes(exclude='object').columns.to_list()

In [40]:
scaler = MinMaxScaler()

x_train_scaled = scaler.fit_transform(train[num_cols])
test_data_scaled = scaler.transform(test[test_num_cols])

In [41]:
train[num_cols] = x_train_scaled
test[test_num_cols] = test_data_scaled

In [42]:
train[cat_cols]

,주구매지점,주구매중분류,주구매대분류,아이여부,고객등급
0,본 점,상품군미지정,가공식품,X,lenith
1,본 점,N.B정장,가공식품,X,MVG-Ace
2,잠실점,N/B골프의류,골프,X,MVG-Prestige
3,부산본점,어덜트,명품,O,lenith
4,인천점,시티웨어,시티웨어,O,lenith
...,...,...,...,...,...
2549,부산본점,상품군미지정,가공식품,X,MVG-Crown
2550,강남점,농산가공,농산물,X,MVG-Crown
2551,본 점,어덜트,명품,X,silver
2552,본 점,건과,농산물,X,MVG-Crown


- 범주형 변수값 제거

In [43]:
# train = train.drop(cat_cols,axis=1)
# test = test.drop(cat_cols,axis=1)

In [44]:
train.shape, test.shape

((2554, 498), (946, 498))

# CatBoostClassifier

In [45]:
from sklearn.model_selection import KFold
!pip install catboost
from catboost import CatBoostClassifier, Pool, cv

cv = KFold(n_splits=5, shuffle=True, random_state=SEED)
params = {'learning_rate': 0.15000000000000002, 'max_depth': 2, 'n_estimators': 200}

model_list = []
score_list = []
for tri,vai in cv.split(train):

    
    # 학습데이터
    x_train = train.iloc[tri]
    y_train = target.iloc[tri] #target
    # 검증데이터
    x_valid = train.iloc[vai]
    y_valid = target.iloc[vai]

    
    # 모델 학습
    model = CatBoostClassifier(**params, random_state=SEED, cat_features=cat_cols) #cat_features=cat_cols
    model.fit(x_train,y_train , eval_set = [(x_valid,y_valid)],early_stopping_rounds=50)

    pred = model.predict_proba(x_valid)[:,1] 
    score = roc_auc_score(y_valid,pred) 
    score_list.append(score) 
    model_list.append(model) 

score_list

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
0:	learn: 0.6663137	test: 0.6673979	best: 0.6673979 (0)	total: 58ms	remaining: 11.6s
1:	learn: 0.6462448	test: 0.6473444	best: 0.6473444 (1)	total: 67ms	remaining: 6.63s
2:	learn: 0.6340064	test: 0.6359216	best: 0.6359216 (2)	total: 75.3ms	remaining: 4.95s
3:	learn: 0.6251728	test: 0.6274532	best: 0.6274532 (3)	total: 84.8ms	remaining: 4.15s
4:	learn: 0.6160816	test: 0.6182546	best: 0.6182546 (4)	total: 93.9ms	remaining: 3.66s
5:	learn: 0.6093634	test: 0.6131896	best: 0.6131896 (5)	total: 102ms	remaining: 3.3s
6:	learn: 0.6041691	test: 0.6066286	best: 0.6066286 (6)	total: 111ms	remaining: 3.05s
7:	learn: 0.6006249	test: 0.6048402	best: 0.6048402 (7)	total: 121ms	remaining: 2.91s
8:	learn: 0.5969152	test: 0.6025820	best: 0.6025820 (8)	total: 130ms	remaining: 2.75s
9:	learn: 0.5911962	test: 0.5968795	best: 0.5968795 (9)	total: 138ms	remaining: 2.62s
10:	learn: 0.5866771	test: 0.5946849	best

[0.7617536318805812,
 0.7598428373188757,
 0.7562824092540885,
 0.7635860615242738,
 0.772400707092003]

In [46]:
pred_list=[]
for model in model_list:
    pred = model.predict_proba(test)[:,1]
    pred_list.append(pred)

In [47]:
pred = np.mean(pred_list,axis=0)
pred.shape

(946,)

In [48]:
sample_submission["target"] = pred
sample_submission

,고객번호,target
0,10001,0.517859
1,10070,0.354702
2,10075,0.324553
3,10086,0.315746
4,10128,0.467247
...,...,...
941,49903,0.446207
942,49918,0.104826
943,49937,0.501206
944,49949,0.267977


In [49]:
sample_submission.to_csv("submit_CBC.csv",index=False)

In [ ]:
# ft_train.to_csv("확인용.csv",index=False)